## Project 1 - John Graunt Team



Our project is focused on the comparison of North Korea and South Korea post-1961, in which differing economic and political systems have had an impact on population statistics. One event in particular that was wanted to study was the North Korean famine in the mid-1990s, which you will see in various parts of our study. 

This initial part of the notebook loads all packages for use in the project. 

In [1]:
!pip install wbdata
!pip install cufflinks==0.17.3
!pip install --quiet wbgapi
!pip install chart_studio
!pip install ipywidgets

import pandas as pd
import wbgapi as wb
import wbdata
import cufflinks as cf
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import ipywidgets
cf.go_offline()

  Using cached wbdata-0.3.0-py3-none-any.whl (14 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.6.2 requires h5py~=3.1.0, but you'll have h5py 3.3.0 which is incompatible.
tensorflow 2.6.2 requires numpy~=1.19.2, but you'll have numpy 1.21.5 which is incompatible.
tensorflow 2.6.2 requires six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you'll have typing-extensions 4.1.1 which is incompatible.
fancyimpute 0.6.0 requires keras==2.4.3, but you'll have keras 2

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [2]:
SOURCE = 40 # "Population estimates and projections
indicators = wbdata.get_indicator(source=SOURCE)
indicators

id                 name
-----------------  -------------------------------------------------------------------
SH.DTH.0509        Number of deaths ages 5-9 years
SH.DTH.1014        Number of deaths ages 10-14 years
SH.DTH.1519        Number of deaths ages 15-19 years
SH.DTH.2024        Number of deaths ages 20-24 years
SH.DTH.IMRT        Number of infant deaths
SH.DTH.IMRT.FE     Number of infant deaths, female
SH.DTH.IMRT.MA     Number of infant deaths, male
SH.DTH.MORT        Number of under-five deaths
SH.DTH.MORT.FE     Number of under-five deaths, female
SH.DTH.MORT.MA     Number of under-five deaths, male
SH.DTH.NMRT        Number of neonatal deaths
SH.DYN.0509        Probability of dying among children ages 5-9 years (per 1,000)
SH.DYN.1014        Probability of dying among adolescents ages 10-14 years (per 1,000)
SH.DYN.1519        Probability of dying among adolescents ages 15-19 years (per 1,000)
SH.DYN.2024        Probability of dying among youth ages 20-24 years (per 1,000)

# [A] Population Statistics


## Population Function

This function named population will deliver information to answer queries regarding year, sex, age, and country.

In [3]:
#Year: xxxx
#sex: Male, Female, All
#age_range: (xx,xx)
#place: Alpha-3 Country Code
def population(year, sex, age_range, place):
    age_high = age_range[1]
    age_low = age_range[0]
    age_high_sep = age_high
    if sex == 'Male' or sex =="Males":
        sex = ".MA"
        return populationagerange(age_low,age_high,sex,place,year)
    elif sex == 'Female' or sex == "Females":
        sex = ".FE"
        return populationagerange(age_low,age_high,sex,place,year)
    else:
        sex = ".MA"
        counter = populationagerange(age_low,age_high,sex,place,year)
        sex = ".FE"
        counter += populationagerange(age_low,age_high,sex,place,year)
        return counter

def populationagerange(age_low, age_high, sex, place, year):
    age_highest = age_high
    if age_high > 80:
        age_high = 79
        df = pd.DataFrame()
        year = str(year)
        low = rounddown5(age_low)
        high = rounddown5(age_high)
        x = range(low+5, age_high, 5)
        dummy = ("SP.POP."+str(low).zfill(2)+str(low+4).zfill(2)+sex)
        counter = wbdata.get_dataframe({dummy:place},country = place).loc[year][0]*(1-age_low%5/5)
        for number in x:
            high = number+4
            dummy = ("SP.POP."+str(number).zfill(2)+str(high).zfill(2)+sex)
            counter += wbdata.get_dataframe({dummy:place},country = place).loc[year][0]
        counter += (wbdata.get_dataframe({("SP.POP.80UP"+sex):place},country = place).loc[year][0])/(age_highest-80)
    else:
        df = pd.DataFrame()
        year = str(year)
        low = rounddown5(age_low)
        high = rounddown5(age_high)
        x = range(low+5, high+1-5, 5)
        counter = wbdata.get_dataframe({("SP.POP."+str(low).zfill(2)+str(low+4).zfill(2)+sex):place},country = place).loc[year][0]*(1-age_low%5/5)
        for number in x:
            high_base = number+4
            dummy = ("SP.POP."+str(number).zfill(2)+str(high_base).zfill(2)+sex)
            counter += wbdata.get_dataframe({dummy:place},country = place).loc[year][0]
        counter += wbdata.get_dataframe({("SP.POP."+str(high).zfill(2)+str(high+4).zfill(2)+sex):place},country = place).loc[year][0]*(age_high%5/5)
    return int(counter)

def rounddown5(x, base=5):
    return base*round((x-2)/base)

In [4]:
population(year=1966,sex='Male',age_range=(0,100),place='USA')

95936568

In [5]:
#assertion test ~ should return an assertion error
#assert population(year=2000,sex='Male',age_range=(0,100),place='WLD') > 7e9/2

This next section is the Population dataframe function, which takes a nation and return a pandas dataframe that is indexed by country and year, with columns representing sex and different age groups. The variable nation takes in either a singular Alpha-3 Country code or a list of numerous country codes. Note that North Korea (Democratic People's Republic of Korea)'s code is 'PRK' and South Korea (Republic of Korea)'s code is 'KOR.'

In [6]:
#Population Dataframe: 

def populationdataframe(nation):
    x = range(5,79,5)
    df = pd.DataFrame()
    df = wbdata.get_dataframe({'SP.POP.0004.MA':'Male  00-04'},country = nation)
    df = df.join(wbdata.get_dataframe({'SP.POP.0004.FE':'Female 00-04'},country = nation))
    df80up = wbdata.get_dataframe({'SP.POP.80UP.MA':'Male 80-UP'}, country = nation).join(
             wbdata.get_dataframe({'SP.POP.80UP.FE':'Female 80-UP'}, country = nation))
    for number in x:
        high = number+4
        dummymale = ("SP.POP."+str(number).zfill(2)+str(high).zfill(2)+".MA")
        dummyfemale = ("SP.POP."+str(number).zfill(2)+str(high).zfill(2)+".FE")
        dummymaletitle = "Males "+str(number).zfill(2)+'-'+str(high).zfill(2)
        dummyfemaletitle = " Female "+str(number).zfill(2)+'-'+str(high).zfill(2)
        dfmale = wbdata.get_dataframe({dummymale:dummymaletitle}, country = nation)
        dffemale = wbdata.get_dataframe({dummyfemale:dummyfemaletitle}, country = nation)
        df = df.join(dfmale, how = 'right')
        df = df.join(dffemale, how = 'right')
    df = df.join(df80up)
    if len(nation)== 1 or isinstance(nation, str):
        df.index = df.index.astype(int)
    else:
        df.index = df.index.set_levels(df.index.levels[1].astype(int),level=1)
    return df
    

The following cell is a Test Function for our two countries of interest: North and South Korea. Note that we don't have data for North Korea in 2021.

In [7]:
#Test Function
countries = {'PRK','KOR'}
trial = populationdataframe(countries)
#trial.index.levels[1].astype(int)
#trial.index = trial.index.set_levels(trial.index.levels[1].astype(int), level=1)
trial

Male  00-04  Female 00-04  Males 05-09  \
country                   date                                           
Korea, Rep.               2021          NaN           NaN          NaN   
                          2020     984018.0      931915.0    1170309.0   
                          2019    1020888.0      967214.0    1177884.0   
                          2018    1066720.0     1010720.0    1179200.0   
                          2017    1111061.0     1052749.0    1175624.0   
...                                     ...           ...          ...   
Korea, Dem. People's Rep. 1964     919927.0      901176.0     837565.0   
                          1963    1011938.0      992074.0     695872.0   
                          1962    1102532.0     1082427.0     571023.0   
                          1961    1107311.0     1088608.0     498411.0   
                          1960     980491.0      965099.0     483881.0   

                                 Female 05-09  Males 10-14   Female 10-14  \
country                   date                                              
Korea, Rep.               2021            NaN          NaN            NaN   
                          2020      1109004.0    1186563.0      1112930.0   
                          2019      1114950.0    1194040.0      1116096.0   
                          2018      1114458.0    1203549.0      1120454.0   
                          2017      1108809.0    1216631.0      1128004.0   
...                                       ...          ...            ...   
Korea, Dem. People's Rep. 1964       834338.0     467877.0       493566.0   
                          1963       700189.0     505053.0       534103.0   
                          1962       582942.0     560866.0       592813.0   
                          1961       516314.0     616810.0       650445.0   
                          1960       505809.0     671168.0       704211.0   

                                Males 15-19   Female 15-19  Males 20-24  \
country                   date                                            
Korea, Rep.               2021          NaN            NaN          NaN   
                          2020    1302129.0      1199049.0    1699918.0   
                          2019    1366353.0      1256284.0    1763876.0   
                          2018    1445941.0      1329359.0    1817067.0   
                          2017    1529880.0      1406225.0    1852058.0   
...                                     ...            ...          ...   
Korea, Dem. People's Rep. 1964     688876.0       718448.0     685064.0   
                          1963     699124.0       728068.0     677432.0   
                          1962     695389.0       723788.0     663967.0   
                          1961     694566.0       722220.0     649312.0   
                          1960     707586.0       733966.0     636451.0   

                                 Female 20-24  ...  Males 60-64  \
country                   date                 ...                
Korea, Rep.               2021            NaN  ...          NaN   
                          2020      1548407.0  ...    1874416.0   
                          2019      1593784.0  ...    1791965.0   
                          2018      1623965.0  ...    1687154.0   
                          2017      1636074.0  ...    1567086.0   
...                                       ...  ...          ...   
Korea, Dem. People's Rep. 1964       710222.0  ...      96947.0   
                          1963       705500.0  ...      94833.0   
                          1962       696554.0  ...      92710.0   
                          1961       688076.0  ...      91808.0   
                          1960       683258.0  ...      92858.0   

                                 Female 60-64  Males 65-69   Female 65-69  \
country                   date                                              
Korea, Rep.               2021            NaN          NaN       

# [B] Population Pyramids



##### Population Pyramids



A python function that takes as input a pandas DataFrame with columns providing counts of people by age-sex groups, and constructs a “population pyramid” graph for visualizing the data.


In [8]:
def populationpyramid(df, year):
    age_ranges = []

    for i in range(0,80,5):
        age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

    age_ranges.append("80UP")
    layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))


    bins = [go.Bar(x = df.loc[year,:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='gold'),
               hoverinfo='skip'
               ),

            go.Bar(x = -df.loc[year,:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='royalblue'),
               hoverinfo='skip',
               )
            ]
    py.offline.iplot(dict(data=bins, layout=layout))

Using the populationdataframe function, we can create a population pyramid for S. Korea.

In [9]:
#Test
dummytrial = populationdataframe('KOR')
populationpyramid(dummytrial, 2020)

Animated population pyramid: A python function that takes as input a pandas DataFrame with columns providing counts of people by age-sex groups, with rows corresponding to different years, and constructs an animated “population pyramid” graph for visualizing how the population changes over time. We have implemented this with a slider function, and aim to smoothen the animation in the next few days.

In [10]:
from ipywidgets import interact
def sliderpyramid(df):
    def pophelper(year):
        #py.init_notebook_mode(connected=True)
        age_ranges = []

        for i in range(0,80,5):
            age_ranges.append(f"{i:02d}"+f"{i+4:02d}")

        age_ranges.append("80UP")
        layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))


        bins = [go.Bar(x = df.loc[year,:].filter(regex="Male").values,
               y = [int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Men',
               marker=dict(color='gold'),
               hoverinfo='skip'
               ),

        go.Bar(x = -df.loc[year,:].filter(regex="Female").values,
               y=[int(s[:2])+1 for s in age_ranges],
               orientation='h',
               name='Women',
               marker=dict(color='royalblue'),
               hoverinfo='skip',
               )
            ]
        py.offline.iplot(dict(data=bins, layout=layout))
    ipywidgets.interact(pophelper, year = (1961,2020,1))

In [11]:
#Interactive Population Pyramid South Korea

sliderpyramid(populationdataframe('KOR'))

interactive(children=(IntSlider(value=1990, description='year', max=2020, min=1961), Output()), _dom_classes=(…

In [12]:
#Interactive Population Pyramid North Korea

sliderpyramid(populationdataframe('PRK'))

interactive(children=(IntSlider(value=1990, description='year', max=2020, min=1961), Output()), _dom_classes=(…

# [C] Additional Visualizations

##### Plotting North and South Korea's Population Over Time

Here, we just wanted an idea of the populations of the two countries over time. You can see that North Korea has always had a lower population, though the gap has widened over time.

In [13]:
#Population Comparison between North and South Korea

variable_labels = {"SP.POP.TOTL":"Population"}

# Three letter codes come from wbdata.get_country()
countries = {"PRK":"North Korea",
             "KOR":"S. Korea"}

df1 = wbdata.get_dataframe(variable_labels, country = countries).squeeze()

df1 = df1.unstack('country')
# Date index is of type string; change to integers
df1.index = df1.index.astype(int)
df1.iplot(title="North and South Korean Population By Year",xTitle='Year',yTitle='Population')

##### Plotting North and South Korea's Population Growth Rates Over Time


In the next few cells, we wanted to take a look at population growth rates between N. Korea, S. Korea, the world, as well as similar high/low income countries in the Eastern Asia Region. We chose Myanmar to compare to North Korea and Japan to compare to South Korea, as they are the closest in terms of GDP per capita to each respective country in Eastern Asia. We see that all 4 countries generally follow the world trend of decreasing population growth rates, though overall higher income countries like Japan and South Korea have lower growth rates that North Korea and Myanmar. However, we note that North Korea's growth rate fluctuates greatly when compared to the others.



In [14]:
import numpy as np

variable_labels = {"SP.POP.TOTL":"Population"}

# Three letter codes come from wbdata.get_country()
countries = {"WLD":"World",
             "PRK":"North Korea",
             "KOR":"S. Korea",
             "JPN":"JPN",
             "MMR":"Myanmar",
            }

df = wbdata.get_dataframe(variable_labels, country = countries).squeeze()

df = df.unstack('country')
# Date index is of type string; change to integers
df.index = df.index.astype(int)

# Differences (over time) in logs give us growth rates
np.log(df).diff().iplot(title="Population Growth Rates of North & South Korea",
                        yTitle="Growth Rate",xTitle='Year')

In [15]:
#Loading Indicators
indicators = {"NY.GDP.PCAP.CD":"GDP per capita",
              "SP.POP.GROW":"Population Growth Rate",
              "SP.DYN.AMRT.MA":"Male Mortality",
              "SP.DYN.AMRT.FE":"Female Mortality",
              "SP.POP.1564.FE.ZS":"% Adult Female",
              "SP.POP.TOTL.FE.ZS":"% Female",
              "SP.POP.1564.MA.ZS":"% Adult Male",
              "SP.POP.TOTL.MA.ZS":"% Male",
              "AG.LND.ARBL.ZS": "Arable land (% of land area)",
              "AG.CON.FERT.PT.ZS": "Fertilizer consumption (% of fertilizer production)",
              "TM.VAL.AGRI.ZS.UN": "Agricultural raw materials imports (% of merchandise imports)",
              "TX.VAL.AGRI.ZS.UN": "Agricultural raw materials exports (% of merchandise exports)",
              "AG.YLD.CREL.KG": "Cereal yield (kg per hectare)"
             }

data = wbdata.get_dataframe(indicators)
data.reset_index(inplace=True)
data['date'] = data['date'].astype(int)
data.set_index(['country','date'],inplace=True)

##### Mortality Rate

We wanted to see the mortality rate between the two countries over time, because we suspected that North Korea's famine had a profound effect on deaths. Looking at the two graphs, we can see that on average, North Korea has always had a higher mortality rate than South Korea, and the spike in the mid-1990s certainly makes correlates with the famine deaths we researched. 

In [16]:
sk = data.query("country=='Korea, Rep.'")
sk.index = sk.index.droplevel('country')

# Drop country index for World data
#sk.index = sk.index.droplevel('country')

sk[["Male Mortality","Female Mortality"]].iplot(title="Deaths per 10,000 in South Korea",yTitle="Deaths per 10,000",xTitle='Year')

In [17]:
nk = data.query('country=="Korea, Dem. People\'s Rep."')
nk.index = nk.index.droplevel('country')
nk[["Male Mortality","Female Mortality"]].iplot(title="Deaths per 10,000 in North Korea",yTitle="Deaths per 10,000",xTitle='Year')

##### Adult male and female share of population over time



Typically, decreases in population growth could also be due to a decreasing share of adult women, perhaps due to gender selection at birth.  We can see from these two graphs that in the 1960s, North Korea had a comparatively higher share of women, though this sharply dropped in the 1970s. This could have been due to the effect of military losses increasing the relative % of women in the 60s. We also note the decrease in the mid-1990s, potentially because of the famine and only recovering in present-day statistics. South Korea gradually increased, and the % difference between female and male is not as pronounced and stayed relatively constant over the last few years.


In [18]:
# % Adult Female is % of females who are adult.
# To make a share of total population take product
sk["% Adult Female"] = sk["% Adult Female"]*sk["% Female"]/100
sk["% Adult Male"] = sk["% Adult Male"]*sk["% Male"]/100

sk[["% Adult Female", "% Adult Male"]].iplot(title="% of Adult Males and Females in South Korean Population",
                                            yTitle="% of Population",xTitle='Year')


In [19]:
nk["% Adult Female"] = nk["% Adult Female"]*nk["% Female"]/100
nk["% Adult Male"] = nk["% Adult Male"]*nk["% Male"]/100

nk[["% Adult Female", "% Adult Male"]].iplot(title="% of Adult Males and Females in North Korean Population",
                                            yTitle="% of Population",xTitle='Year')

##### Imports and Exports

We attempted to analyze agricultural imports and exports as a % of total, and we can see that North Korea doesn't have any data on this. We wanted to take a look at this because it might tell us clues about why the famine occurred (e.g., reduced imports after the collapse of the Soviet Union).

In [20]:
sk[["Agricultural raw materials imports (% of merchandise imports)",
    "Agricultural raw materials exports (% of merchandise exports)"]].iplot(title="Agricultural imports and exports as a % of total",
                                            yTitle="% of total imports/exports",xTitle='Year')

In [21]:
nk[["Agricultural raw materials imports (% of merchandise imports)",
    "Agricultural raw materials exports (% of merchandise exports)"]].iplot(title="Agricultural imports and exports as a % of total",
                                            yTitle="% of total imports/exports",xTitle='Year')

##### Cereal Yield

We also decided cereal yield would be an important factor to look at in terms of population, as it forms the bulk of staple foods for most populations. We can see that this is one clear area in which South Korea performs strongly and consistently in, whereas in North Korea, we can see that it reaches a peak of around 6000kg/ha right before a dramatic decrease in the mid 1990s. While South Korea continued to increase its cereal yield to almost 7000kg/ha, North Korea never recovered and its current yields are around 4500kg/ha. 

In [22]:
sk["Cereal yield (kg per hectare)"].iplot(title="Cereal Yield in South Korea",
                                            yTitle="kg per hectacre",xTitle='Year')

In [23]:
nk["Cereal yield (kg per hectare)"].iplot(title="Cereal Yield in North Korea",
                                            yTitle="kg per hectacre",xTitle='Year')

##### Arable Land

The last factor we will look at for now is arable land. This is defined as the amount of land available for the growth of crops. We see that North Korea is gradually increasing its arable land, whereas South Korea's has been steadily decreasing. So, we think that the food insecurity suffered by North Koreans over the last 2-3 decades may be due to specific input factors, which we will take a look at. 

In [24]:
sk["Arable land (% of land area)"].iplot(title="Arable Land in South Korea",
                                            yTitle="% of Land Area",xTitle='Year')

In [25]:
nk["Arable land (% of land area)"].iplot(title="Arable Land in North Korea",
                                            yTitle="% of Land Area",xTitle='Year')

##### Additional Agriculture Analysis

In [26]:
!pip install eep153_tools
!pip install python_gnupg

from eep153_tools.sheets import decrypt_credentials
decrypt_credentials('students.json.gpg')

  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
  Using cached python_gnupg-0.4.8-py2.py3-none-any.whl (18 kB)
Input secret passphrase for students.json.gpg to create google drive credentials: noodle octopus


In [27]:
!ls ~/.eep153.service_accounts/

students@eep153.iam.gserviceaccount.com


In [28]:
from eep153_tools.sheets import read_sheets

#### Read a bunch of google worksheets into a dictionary of dataframes
data = read_sheets('https://docs.google.com/spreadsheets/d/1DLn9owcS7ggojJGWlI9vKSz0hqozn6cbcqNGWgzMZ8k',force_numeric=False)

data.keys()

Key available for students@eep153.iam.gserviceaccount.com.


dict_keys(['Explanation', 'Ag TFP', 'Output', 'Inputs', 'Factor Shares', 'Ag Land', 'Land Weights', 'Cropland', 'Irrig', 'Pasture', 'Labor', 'Livestock', 'Machinery', 'Fertilizer', 'Feed'])

In [29]:
data['Output'].head()

,Gross Agricultural Production,,,,,,,,,,...,,,,,,,,,,
0,"($1000, constant 2004-2006 prices)",,,,,,,,,,...,,,,,,,,,,
1,Order,FAO N,WDI Code,Country,Region,Sub-Region,Inc I,Inc II,Note,,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
2,1,202,ZAF,South Africa,"AFRICA, Developed",South Africa,MI-U,MI-U-xC,,,...,-0.010,0.033,0.022,0.059,-0.023,-0.042,,,,
3,2,32,CMR,Cameroon,"AFRICA, SUB-SAHARA","SSA, Central",MI-L,MI-L,,,...,0.028,0.055,0.027,0.008,0.046,0.061,,,,
4,3,37,CAF,Central African Republic,"AFRICA, SUB-SAHARA","SSA, Central",LI,LI,,,...,0.050,0.016,0.011,-0.048,0.024,0.020,,,,


In [30]:
import pandas as pd

def get_international_ag_productivity_data(series):

    columns=series.iloc[1,:]  # Need to fix columns
    series = series.iloc[2:,:]
    series.columns = columns

    # Set index to country code
    series = series.set_index('WDI Code')

    series = series[['%d' % t for t in range(1961,2020)]]

    series = series.replace({',':''},regex=True) # Get rid of commas in number strings
    series = series.replace({'':'NaN'}) # Change empty cells to NaN strings
    series = series.apply(lambda x: pd.to_numeric(x,errors='ignore'))
    series = series.astype(float) # Convert to floats

    series = series.stack()
    
    series = series.loc[~series.index.duplicated(keep='first')] 

    series.index.names = ['WDI Code','Year']

    return series
   

Data = ['Output','Ag TFP','Ag Land','Irrig','Pasture','Labor','Livestock',
        'Machinery','Fertilizer','Feed']

D = {}
for key in Data:
    D[key] = get_international_ag_productivity_data(data[key])

df = pd.DataFrame(D)
df

Output  Ag TFP  Ag Land  Irrig  Pasture   Labor  Livestock  \
WDI Code Year                                                                  
         1961   441067.0   111.0    184.0    4.0     56.0    31.0      263.0   
         1962   449727.0   113.0    184.0    4.0     56.0    30.0      252.0   
         1963   455937.0   111.0    189.0    4.0     54.0    30.0      259.0   
         1964   441651.0   108.0    187.0    4.0     59.0    30.0      253.0   
         1965   484620.0   121.0    188.0    5.0     61.0    29.0      246.0   
...                  ...     ...      ...    ...      ...     ...        ...   
ZWE      2015  1419365.0    97.0   2460.0  175.0  12100.0  4877.0     5278.0   
         2016  1515900.0    91.0   2966.0  175.0  12100.0  5023.0     5931.0   
         2017        NaN     NaN      NaN    NaN      NaN  5169.0        NaN   
         2018        NaN     NaN      NaN    NaN      NaN  5330.0        NaN   
         2019        NaN     NaN      NaN    NaN      NaN  5499.0        NaN   

               Machinery  Fertilizer       Feed  
WDI Code Year                                    
         1961     1692.0      2358.0   134016.0  
         1962     1740.0      2496.0   132329.0  
         1963     1890.0      2814.0   157105.0  
         1964     1976.0      3327.0   159653.0  
         1965     1679.0      3537.0   140357.0  
...                  ...         ...        ...  
ZWE      2015    31350.0     91600.0  1318934.0  
         2016    32911.0    130000.0  1501953.0  
         2017        NaN         NaN        NaN  
         2018        NaN         NaN        NaN  
         2019        NaN         NaN        NaN  

[10974 rows x 10 columns]

We can see that there is a massive increase in machinery in South Korea, whereas in North Korea, it stays nearly constant, though with a drop in the mid-1990s that may explain the famine. These are concentrated most importantly in fertilizer and feed.

In [31]:
select = df.xs('KOR',level='WDI Code').dropna(how='any')

select = select/select.loc['1961',:]
select.iplot(title='South Korea')

In [32]:
select = df.xs('KOR',level='WDI Code').dropna(how='any').drop("Machinery", 1)

select = select/select.loc['1961',:]
select.iplot(title='South Korea')

/tmp/ipykernel_42/1724169444.py:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



This is a graph depicting the variety of agricultural factors in South Korea. As you can see, it’s pretty different compared to the North Korea graph. Almost all of the agricultural elements are zero, except for machinery, which is indicated by the green line on the left. The reason behind this could be due to the fact that less than one-fourth of the republic’s area is cultivated and South Korea relies heavily on imports. The natural resources required for agriculture in South Korea are not abundant and arable land only accounts for 22 percent of the country's land. The machinery line increased steadily from 1972 and plateaus around 2000 because the Korean 

In [33]:
select = df.xs('PRK',level='WDI Code').dropna(how='any')

select = select/select.loc['1961',:]
select.iplot(title='North Korea')

Above is a line graph that shows different agriculture factors in North Korea. Out of all of the lines, I think the most important ones to note are fertilizer, the red/orangish line, and machinery, the green line. As you can see in this graph, there was an increase in fertilizer over time until around 1977. At that point, fertilizer fluctuated a bit until 1993. From there, there was an extremely significant decline in fertilizer. This drop in fertilizer coincided with North Korea announcing a 3-year transitional economic policy placing primary emphasis on agriculture, light industry, and foreign trade. A lack of fertilizer, natural disasters, and poor storage and transportation practices left the country more than a million tons per year short of grain self-sufficiency. All of these factors led to North Korea’s famine, which was a period of mass starvation together with a general economic crisis that lasted from 1994 to 1998. If you look closely at the feed line, which is represented by the light blue line, you can see that it mimics the fertilizer trend as well. It’s not as extreme and apparent, but it definitely follows a similar pattern. Mechanization is another agricultural target that’s important and represented by the green line. You can see that there is a plateau in the green line starting from 1984. This is because by 1984, mechanization had reached the level of seven tractors per 100 hectares in the plains and six tractors per 100 hectares in the intermediate and mountainous areas. These ratios are quoted again in official pronouncements of the early 1990s, which indicates that there was no further advancement in mechanized agriculture since then. This aligns with our data, as we can see that the green line plateaus after 1984.


In [34]:
SK = df.xs('KOR',level='WDI Code').dropna(how='any')

NK = df.xs('PRK',level='WDI Code').dropna(how='any')

select = np.log(NK/SK)
select.iplot(title='Log N. Korea inputs & output Relative to S. Korea')